In [ ]:
# 📊 Model Evaluation and Metrics – Titanic Dataset
# This notebook demonstrates key classification metrics using Scikit-learn

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


ModuleNotFoundError: No module named 'matplotlib'

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, classification_report
)

ModuleNotFoundError: No module named 'sklearn'

In [23]:

# ## Objectifs :
# - Préprocessing des données
# - Comparaison de modèles
# - Feature engineering
# - Cross-validation
# - Interface Streamlit

# %%
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import warnings
warnings.filterwarnings('ignore')

# Configuration pour les graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# %%
# Chargement des données
print("Chargement du dataset Titanic...")
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

print(f" Dataset chargé avec succès!")
print(f" Forme du dataset: {df.shape}")
print(f" Taux de survie global: {df['Survived'].mean():.2%}")

# %%
# Exploration des données
print(" EXPLORATION DES DONNÉES")
print("="*50)
df.head()

# %%
# Informations sur le dataset
print("ℹ Informations sur le dataset:")
df.info()

# %%
# Statistiques descriptives
print(" Statistiques descriptives:")
df.describe()

# %%
# Valeurs manquantes
print("Valeurs manquantes:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Valeurs manquantes': missing_data,
    'Pourcentage': missing_percent
}).sort_values('Valeurs manquantes', ascending=False)

print(missing_df[missing_df['Valeurs manquantes'] > 0])

# %%
# Visualisations exploratoires
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Survie par classe
sns.countplot(data=df, x='Pclass', hue='Survived', ax=axes[0,0])
axes[0,0].set_title('Survie par Classe')
axes[0,0].legend(['Non survivant', 'Survivant'])

# Survie par sexe
sns.countplot(data=df, x='Sex', hue='Survived', ax=axes[0,1])
axes[0,1].set_title('Survie par Sexe')
axes[0,1].legend(['Non survivant', 'Survivant'])

# Distribution des âges
df['Age'].hist(bins=30, ax=axes[1,0], alpha=0.7)
axes[1,0].set_title('Distribution des Âges')
axes[1,0].set_xlabel('Âge')

# Prix des billets
df['Fare'].hist(bins=30, ax=axes[1,1], alpha=0.7)
axes[1,1].set_title('Distribution des Prix')
axes[1,1].set_xlabel('Prix du billet')

plt.tight_layout()
plt.show()

# %%
# 🧹 Sélection des features
print("🧹 SÉLECTION DES FEATURES")
print("="*40)

features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
target = 'Survived'

X = df[features].copy()
y = df[target]

print(f"Features sélectionnées: {features}")
print(f"Target: {target}")
print(f"Forme de X: {X.shape}")

# %%
# 🔧 Pipeline de preprocessing
print("🔧 CRÉATION DU PIPELINE DE PREPROCESSING")
print("="*50)

# Features numériques et catégorielles
numeric_features = ['Age', 'Fare']
categorical_features = ['Pclass', 'Sex', 'Embarked']

print(f"Features numériques: {numeric_features}")
print(f"Features catégorielles: {categorical_features}")

# Transformers
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Preprocessor complet
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

print("Pipeline de preprocessing créé!")

# %%
# 🤖 MODÈLE BASELINE - LOGISTIC REGRESSION
print("MODÈLE BASELINE - LOGISTIC REGRESSION")
print("="*50)

# Pipeline complet
clf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Division des données
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Taille train: {X_train.shape[0]}")
print(f"Taille test: {X_test.shape[0]}")

# Entraînement
clf_pipeline.fit(X_train, y_train)

# Prédictions
y_pred_lr = clf_pipeline.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

print(f"\nRésultats Logistic Regression:")
print(f"Accuracy: {accuracy_lr:.4f}")
print("\nRapport de classification:")
print(classification_report(y_test, y_pred_lr))

# %%
# 🌳 EXERCICE 1 - RANDOM FOREST
print("EXERCICE 1 - RANDOM FOREST CLASSIFIER")
print("="*50)

# Pipeline Random Forest
rf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entraînement
rf_pipeline.fit(X_train, y_train)

# Prédictions
y_pred_rf = rf_pipeline.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"Résultats Random Forest:")
print(f"Accuracy: {accuracy_rf:.4f}")
print("\nRapport de classification:")
print(classification_report(y_test, y_pred_rf))

# Comparaison
print(f"\nCOMPARAISON:")
print(f"Logistic Regression: {accuracy_lr:.4f}")
print(f"Random Forest: {accuracy_rf:.4f}")
print(f"Amélioration: {accuracy_rf - accuracy_lr:+.4f}")

# %%
# Matrice de confusion
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Logistic Regression
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', ax=axes[0], cmap='Blues')
axes[0].set_title('Logistic Regression')
axes[0].set_xlabel('Prédiction')
axes[0].set_ylabel('Réalité')

# Random Forest
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', ax=axes[1], cmap='Greens')
axes[1].set_title('Random Forest')
axes[1].set_xlabel('Prédiction')
axes[1].set_ylabel('Réalité')

plt.tight_layout()
plt.show()

# %%
# 🔄 EXERCICE 2 - CROSS-VALIDATION
print("🔄 EXERCICE 2 - CROSS-VALIDATION")
print("="*50)

# Cross-validation pour Logistic Regression
cv_scores_lr = cross_val_score(clf_pipeline, X, y, cv=5, scoring='accuracy')
print(f"🔵 Logistic Regression CV:")
print(f"Scores: {cv_scores_lr}")
print(f"Moyenne: {cv_scores_lr.mean():.4f} (±{cv_scores_lr.std() * 2:.4f})")

# Cross-validation pour Random Forest
cv_scores_rf = cross_val_score(rf_pipeline, X, y, cv=5, scoring='accuracy')
print(f"\n🟢 Random Forest CV:")
print(f"Scores: {cv_scores_rf}")
print(f"Moyenne: {cv_scores_rf.mean():.4f} (±{cv_scores_rf.std() * 2:.4f})")

# Analyse de stabilité
print(f"\n📈 ANALYSE DE STABILITÉ:")
print(f"Logistic Regression - Écart-type: {cv_scores_lr.std():.4f}")
print(f"Random Forest - Écart-type: {cv_scores_rf.std():.4f}")

if cv_scores_lr.std() < cv_scores_rf.std():
    print("✅ Logistic Regression est plus stable")
else:
    print("✅ Random Forest est plus stable")

# %%
# 📊 Visualisation des scores CV
plt.figure(figsize=(10, 6))
cv_data = pd.DataFrame({
    'Logistic Regression': cv_scores_lr,
    'Random Forest': cv_scores_rf
})

cv_data.boxplot()
plt.title('Distribution des Scores de Cross-Validation')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.show()

# %%
# 🔧 EXERCICE 3 - FEATURE ENGINEERING
print("🔧 EXERCICE 3 - FEATURE ENGINEERING")
print("="*50)

# Ajouter FamilySize
df['FamilySize'] = df['SibSp'] + df['Parch']
print("✅ Feature FamilySize créée (SibSp + Parch)")

# Distribution de FamilySize
print(f"\n📊 Distribution de FamilySize:")
print(df['FamilySize'].value_counts().sort_index())

# Visualiser FamilySize vs Survie
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='FamilySize', hue='Survived')
plt.title('Survie par Taille de Famille')
plt.xlabel('Taille de Famille')
plt.legend(['Non survivant', 'Survivant'])
plt.show()

# %%
# 🔄 Modèles avec feature engineering
print("🔄 MODÈLES AVEC FEATURE ENGINEERING")
print("="*50)

# Features enrichies
features_enhanced = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize']
X_enhanced = df[features_enhanced].copy()

# Nouveau preprocessor
numeric_features_enhanced = ['Age', 'Fare', 'FamilySize']
categorical_features_enhanced = ['Pclass', 'Sex', 'Embarked']

preprocessor_enhanced = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_features_enhanced),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
    ]), categorical_features_enhanced)
])

# Pipelines enrichis
clf_pipeline_enh = Pipeline([
    ('preprocessing', preprocessor_enhanced),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

rf_pipeline_enh = Pipeline([
    ('preprocessing', preprocessor_enhanced),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# %%
# 🎯 Entraînement et évaluation des modèles enrichis
X_train_enh, X_test_enh, y_train_enh, y_test_enh = train_test_split(
    X_enhanced, y, test_size=0.2, random_state=42, stratify=y
)

# Entraînement
clf_pipeline_enh.fit(X_train_enh, y_train_enh)
rf_pipeline_enh.fit(X_train_enh, y_train_enh)

# Prédictions
y_pred_lr_enh = clf_pipeline_enh.predict(X_test_enh)
y_pred_rf_enh = rf_pipeline_enh.predict(X_test_enh)

# Accuracies
accuracy_lr_enh = accuracy_score(y_test_enh, y_pred_lr_enh)
accuracy_rf_enh = accuracy_score(y_test_enh, y_pred_rf_enh)

print(f"📊 RÉSULTATS AVEC FEATURE ENGINEERING:")
print(f"Enhanced Logistic Regression: {accuracy_lr_enh:.4f}")
print(f"Enhanced Random Forest: {accuracy_rf_enh:.4f}")

# %%
# 📈 COMPARAISON FINALE
print("📈 COMPARAISON FINALE")
print("="*60)

results_df = pd.DataFrame({
    'Modèle': [
        'Logistic Regression (Original)',
        'Random Forest (Original)',
        'Logistic Regression (Enhanced)',
        'Random Forest (Enhanced)'
    ],
    'Test Accuracy': [accuracy_lr, accuracy_rf, accuracy_lr_enh, accuracy_rf_enh],
    'CV Mean': [cv_scores_lr.mean(), cv_scores_rf.mean(), 'N/A', 'N/A']
})

print(results_df.to_string(index=False, float_format='%.4f'))

# Impact du feature engineering
print(f"\n🚀 IMPACT DU FEATURE ENGINEERING:")
print(f"Logistic Regression: {accuracy_lr_enh - accuracy_lr:+.4f}")
print(f"Random Forest: {accuracy_rf_enh - accuracy_rf:+.4f}")

# %%
# 💾 Sauvegarde du meilleur modèle
best_accuracy = max(accuracy_lr, accuracy_rf, accuracy_lr_enh, accuracy_rf_enh)
if best_accuracy == accuracy_rf_enh:
    best_model = rf_pipeline_enh
    best_name = "Random Forest (Enhanced)"
elif best_accuracy == accuracy_lr_enh:
    best_model = clf_pipeline_enh
    best_name = "Logistic Regression (Enhanced)"
elif best_accuracy == accuracy_rf:
    best_model = rf_pipeline
    best_name = "Random Forest (Original)"
else:
    best_model = clf_pipeline
    best_name = "Logistic Regression (Original)"

print(f"MEILLEUR MODÈLE: {best_name}")
print(f" Accuracy: {best_accuracy:.4f}")

# Sauvegarder
joblib.dump(best_model, "titanic_best_model.pkl")
print(f"💾 Modèle sauvegardé: titanic_best_model.pkl")

# %%
# 🎉 RÉSUMÉ FINAL
print("🎉 ANALYSE TERMINÉE!")
print("="*60)
print(f"✅ Meilleur modèle: {best_name}")
print(f"📊 Performance: {best_accuracy:.4f}")
print(f"🔧 Feature Engineering: {'Oui' if 'Enhanced' in best_name else 'Non'}")
print(f"💾 Modèle prêt pour le déploiement!")

# %% [markdown]
# ## 🎯 Conclusions
# 
# 1. **Random Forest** performe généralement mieux que **Logistic Regression**
# 2. **Feature Engineering** (FamilySize) peut améliorer les performances
# 3. **Cross-validation** montre la stabilité des modèles
# 4. Le modèle est prêt pour le déploiement avec **Streamlit**
# 
# ### 🚀 Prochaines étapes :
# - Créer l'interface Streamlit
# - Tester d'autres features (Title, Deck, etc.)
# - Optimiser les hyperparamètres
# - Déployer en production

ModuleNotFoundError: No module named 'matplotlib'